In [1]:
import re
import glob
from tqdm.auto import tqdm

In [2]:
sports = ["Football", "Soccer", "Basketball", "Cricket", "Tennis", "Golf", "Rugby", "Baseball", "American Football", "Hockey", "Athletics", "Boxing", "Martial Arts", "Cycling", "Swimming", "Table Tennis", "Volleyball", "Wrestling", "Skiing", "Snowboarding", "Surfing", "Gymnastics", "Badminton", "Formula 1", "Horse Racing", "Sailing", "Archery", "Canoeing", "Equestrian", "Fencing", "Handball", "Judo", "Karate", "Rowing", "Squash", "Taekwondo", "Triathlon", "Weightlifting", "Bobsleigh", "Bouldering", "Curling", "Darts", "Esports", "Field Hockey", "Ice Skating", "Lacrosse", "Polo", "Skateboarding", "Snooker", "Softball", "Water Polo"]
sports_orgs = ["ESPN", "BBC Sport", "Bleacher Report", "Sports Illustrated", "Sky Sports", "Fox Sports", "CBS Sports", "Yahoo Sports", "NBC Sports", "The Athletic", "ESPNcricinfo", "Eurosport", "SB Nation", "Deadspin", "The Score", "The Guardian Sport", "FourFourTwo", "Goal.com", "The Ringer", "Sporting News", "Athletic Bilbao", "The Independent Sport", "Sportsnet", "Talksport", "ESPN FC", "USA Today Sports", "MLB.com", "NFL.com", "NBA.com", "NHL.com", "PGA Tour", "Tennis.com", "Cyclingnews", "World Rugby", "UFC News", "Boxing Scene", "FloWrestling", "Runner's World", "F1.com", "Olympic Channel", "NBC Olympics", "Surfer Magazine", "Transworld Skateboarding", "Golf Digest", "PokerNews", "International Gymnast", "Swimming World", "Track and Field News", "Badminton World Federation"]
#sports_words = ["athlete", "sports", "sport", "team", "game", "match", "competition", "tournament", "score", "goal", "victory", "defeat", "champion", "coach", "training", "practice", "fitness", "exercise", "athlete's foot", "jersey", "equipment", "ball", "stadium", "arena", "field", "pitch", "race", "league", "referee", "umpire", "halftime", "overtime", "sportsmanship", "spectators", "cheerleader", "medal", "trophy", "athleticism", "championship", "drills", "injury", "recovery", "strategy", "playbook", "endorsement", "contract", "merchandise"]

In [3]:
all_sports = sports + sports_orgs # + sports_words
all_sports = [sport.lower() for sport in all_sports]

In [4]:
files = []
for filename in glob.iglob('/users/ujan/sports-language-in-politics/data/raw/now_2022/'+'**/*.txt', recursive=True):
    files.append(filename)

In [5]:
months = ['0'+str(i) for i in range(1, 10)]
months.extend(['11', '12'])

In [ ]:
sample_limit = 50000
sports_samples = []
non_sports_samples = []
bar = tqdm(range(12), position=0)
# for each month
for month in months:
    
    data_files = []
    source_file = None
    # for each month get all data files and source file
    for file in files:
        if 'now-22-'+month in file:
            if 'sources' not in file:
                data_files.append(file)
            else:
                source_file = file
    # filenames extracted for each month
    # now get data from the files
    data = []
    sources = []
    # get source file data for the month
    with open(source_file, errors='replace') as f:
        for line in f:
            sources.append(line)
    # get source ids
    # source_id : source link
    source_ids = [int(source.split('\t')[0]) for source in sources]
    source_dict = {source_ids[i]: sources[i] for i in range(len(sources))}
    # get data for each data file for the month
    month_bar = tqdm(range(len(data_files)), position=1)
    for file in data_files:
        with open(file) as f:
            for line in f:
                data.append(line)
        # first line is empty
        data = data[1:]
        # get data ids
        # data_id : data
        new_data = []
        data_ids = []
        for d in data:
            if len(d.split()) < 2:
                continue
            if not d.split()[0][2:].isdigit():
                continue
            data_ids.append(int(d.split()[0][2:]))
            new_data.append(' '.join(d.split()[1:]))
        data_dict = {data_ids[i]: new_data[i] for i in range(len(new_data))}
        for id in data_ids:
            sample = data_dict[id]
            if id in source_dict:
                source = source_dict[id]
                matches = set(re.findall(r"(?=("+'|'.join(all_sports)+r"))", source_dict[id]))
                if len(matches) == 0:
                    non_sports_samples.append(sample)
                else:
                    sports_samples.append(sample)
        month_bar.update(1)

    month_bar.refresh()
    month_bar.reset()
    bar.update(1)
    if len(sports_samples) > sample_limit and len(non_sports_samples) > sample_limit:
        print('sample limit reached')
        break

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

In [ ]:
bar = tqdm(range(len(sports_samples)), position=0)
sports_samples_clean = []
for sample in sports_samples:
    clean_sample = re.sub(r"[^a-zA-Z0-9]+", ' ', sample).lower()
    sports_samples_clean.append(clean_sample)
    bar.update(1)

In [ ]:
bar = tqdm(range(len(non_sports_samples)), position=0)
non_sports_samples_clean = []
for sample in non_sports_samples:
    clean_sample = re.sub(r"[^a-zA-Z0-9]+", ' ', sample).lower()
    non_sports_samples_clean.append(clean_sample)
    bar.update(1)